# Light Curve Morphology

This notebook inspects the shapes of simulated light curves for a variety of custom models.


In [ ]:
import sys
from copy import deepcopy
from pathlib import Path

from astropy.table import Table, vstack
import numpy as np
import sncosmo
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator

sys.path.insert(0, '../')
from analysis import models

figure_output_dir = Path('./figures/light_curve_morphology').resolve()
figure_output_dir.mkdir(exist_ok=True, parents=True)
models.register_sources(force=True)


## Comparing Light Curve Shapes

We start by comparing the shapes of each light-curve model in each band.

In [ ]:
def calculate_flux(model, phase, filt):
    """Calculate magnitudes in a given filter
    
    This function is a wrapper for ``model.bandflux`` and
    returns NAN values for phases that would otherwise 
    raise a ValueError for being out of the model range.
    
    Args:
        model    (Model): An sncosmo model
        phase    (array): An array of phase values
        filt       (str): Name of an scnosmo registered filter
        
    Returns:
        An array of AB fluxes relative to a ZP of 25
    """

    try:
        flux = model.bandflux(filt, phase, zpsys='AB', zp=25)

    except ValueError:
        flux = []
        for p in phase:
            try:
                flux.append(model.bandflux(filt, p, zpsys='AB', zp=25))

            except ValueError:
                flux.append(np.nan)


    return np.ma.array(flux, mask=np.isnan(flux))

def plot_light_curves(sources, phase, offset=1):
    """Plot the normalized flux in LSST bands for a set of models
    
    Args:
        sources  (list): A list of sncosmo sources
        phase (ndarray): An array of phase values
        offset  (float): Vertical offset between light curves (default = 1)
    
    Returns:
        A matplotlib figure
        An array of figure axes
    """
    
    bands = ['lsst' + b for b in 'ugrizy']
    fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharex=True, sharey=True)
    for i, source in enumerate(sources):
        model = sncosmo.Model(source)
        t0 = source.peakphase('standard::b')

        for axis, band in zip(axes.flatten(), bands):        
            flux = calculate_flux(model, phase + t0, band) 
            flux = (flux / max(flux)) + (i * offset)
            axis.plot(phase, flux, label=model.source.name)
            axis.set_title(band)

    axis.legend()
    for axis in axes[-1]:
        axis.set_xlabel('phase')

    for axis in axes[:, 0]:
        axis.set_ylabel('Normalized Flux')
    
    plot_height = offset * (len(sources) - 1) + ((2 + offset) / 2)
    axis.set_ylim(0, plot_height)
    
    return fig, axes


In [ ]:
phase = np.arange(-20, 130.5, .5)
sources = [sncosmo.get_source('CMFGEN', version=v) for v in (1.02, 1.04, 1.4, 1.7)]
sources += [sncosmo.get_source('salt2'), sncosmo.get_source('hsiao')]
fig, axes = plot_light_curves(sources, phase, offset=.5)
plt.savefig(figure_output_dir / 'model_morphology.pdf')
plt.show()


## Representative "stretch" and "color"

We wish to determine fiducial "stretch" and "color" values for our custom models. We do this by simulating light curves with each model, fitting each those light-curves with a salt2-like model, and then plotting the relationship between the two sets of parameters. 

First we create a set table of simulated observations that defines the cadence of our simulated light curves.


In [ ]:
time = np.arange(0, 130, 3).tolist()

bands = ['lsst' + b for b in 'ugrizy']
observations = vstack([
    Table(
        {'time': time,
         'band': np.full(len(time), band),
         'gain': np.ones(len(time)),
         'skynoise': np.zeros(len(time)),
         'zp': np.full(len(time), 25),
         'zpsys': np.full(len(time), 'ab')})
    
    for band in bands])

observations[:10]


Next, we run a set of simulations and generate a single light-curve for each custom model. Note that the cadence for each of these simulated light curves is exactly the same.

In [ ]:
light_curves = []
custom_sources = [sncosmo.get_source('CMFGEN', version=v) for v in (1.02, 1.04, 1.4, 1.7)]
for source in custom_sources:
    model = sncosmo.Model(source)
    params = [{'z': 0, 'x0': 1}]
    lc = sncosmo.realize_lcs(observations, model, params)[0]
    lc.meta['source'] = model.source.name
    lc.meta['version'] = model.source.version
    light_curves.append(lc)
    
light_curves[0][:10]


Each simulated light-curve is then fit with the salt2 model.

In [ ]:
salt2 = sncosmo.Model('salt2')
salt2.set(z=0)

params_list = []
for lc in light_curves:
    result, fitted_model = sncosmo.fit_lc(lc, salt2, ['t0', 'x0', 'x1', 'c'])
    source_name = lc.meta['source']
    source_version = lc.meta['version'] 

    result_dict = {p:v for p, v in zip(result.param_names, result.parameters)}
    result_dict['source'] = source_name
    result_dict['version'] = source_version
    params_list.append(result_dict)
    
    print(r'{} M={}'.format(source_name, source_version))
    sncosmo.plot_lc(lc, fitted_model)
    plt.savefig(figure_output_dir / f'{source_name}_{source_version}_s2fit.pdf')
    plt.show()

salt2_parameters = Table(rows=params_list)

# Reorder table for easy reading
salt2_parameters = salt2_parameters['source', 'version', 'x0', 'x1', 'c']  
salt2_parameters
